In [1]:
import time
start_time = time.time()
import numpy as np
import os
import SimpleITK as sitk
import json
import ProstateLesionDetectionUtils
from scipy import ndimage
import tensorflow as tf
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, (np.int_, np.intc, np.intp, np.int8,
            np.int16, np.int32, np.int64, np.uint8,
            np.uint16, np.uint32, np.uint64)):
            return int(obj)
        elif isinstance(obj, (np.float_, np.float16, np.float32, 
            np.float64)):
            return float(obj)
        elif isinstance(obj, (np.ndarray,)): 
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

with open("ProstateNetData/Datasets_Updated.json", 'r') as f:
    datasets = json.load(f)
prostatenet = "ProstateNetData/Dataset013_ProstateNet_Not_Integerized"
prostatenet = r"E:\DIMITRIS_DEEP_LEARNING\DATASETS\PROSTATENET_UC1_LESIONS\nnUNet_raw\Dataset011_ProstateNet"
with open(r"folds.json", 'r') as f:
    ValidationFolds = json.load(f)
datasets
for dataset in datasets.keys():
    for pat in datasets[dataset].keys():
        for seq in datasets[dataset][pat].keys():
            x = datasets[dataset][pat][seq]
            x = x.replace("\\","/")
            datasets[dataset][pat][seq] = x

2023-09-14 12:03:02.762806: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-14 12:03:03.638001: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/medadmin/anaconda3/envs/tf12/lib/python3.10/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [2]:
loader = ProstateLesionDetectionUtils.DatasetsLoaders.Nifti_Loader.LoadSitkImages(ptdc = datasets["ProstateNet_Not_Integerized"], folder = prostatenet)
patients = loader.load_sitkobj()
patients_test = {key:patients[key] for key in patients.keys() if key in ValidationFolds["Fold 0"]}

In [13]:
model_names = ['Unet', 'Attention_Unet', 'Vnet', 'USE_net', 'Dense2_Unet', 'Trans_Unet']
base_path = "EXPERIMENTS_V0.2"
fold = "fold_0"
weights = "Final_1024.tf"
weights_check = "Checkpoint_1024.tf"


for md_nm in model_names:
    
    path_parent = [base_path, md_nm, fold]
    paths = [base_path, md_nm, fold,"Models", weights] # chains the paths
    model = tf.keras.models.load_model(os.path.join(*paths),compile = False)
    print(model.count_params())

    eval_params = {
        "MODEL": model,
        "EVALUATION_DATA": patients_test,# Needs to be a dictionary {PatName:{T2:arr, ADC:arr, DWI:arr, "Lesion":arr}}
        "SAVE_FOLDER":os.path.join(*path_parent)
    }
    STEPS_PER_EPOCH = len(Xtest) // 2
    INIT_LR = 0.0001
    MAX_LR = 0.05
    CLR = tfa.optimizers.CyclicalLearningRate(initial_learning_rate=INIT_LR,
    maximal_learning_rate=MAX_LR,
    scale_fn=lambda x: 1/(2.**(x-1)),
    step_size=2 * STEPS_PER_EPOCH)
    LOSS = ProstateLesionDetectionUtils.Evaluation.Loss_functions.combined_loss#tfa.losses.sigmoid_focal_crossentropy  #tfa.losses.sigmoid_focal_crossentropy
    OPTIMIZER = tf.keras.optimizers.legacy.Adam(CLR)
    METRICS = [ProstateLesionDetectionUtils.Evaluation.Loss_functions.dice_coefficient]
    eval_params["MODEL"].compile(optimizer=OPTIMIZER,
    loss=LOSS,
    metrics=METRICS)
    eval = ProstateLesionDetectionUtils.Evaluation.Evaluation.Evaluation(eval_params)
    eval.predict()
    #eval.calculate_detorient_metric()
    eval.save_gt()
    eval.save_predictions()
    eval.save_predictions_binary()
    metrics = eval.get_metrics()
    preds = eval.get_predictions()
    
    for key,value in metrics.items():
        metrics[key][0].update({"Model": md_nm, "Number of Parameters":[model.count_params()]})

    path_parent.append("Metrics_Json")
    path_parent.append(f"metrics_final_{paths[-1][:-3]}.json")
    
    with open(os.path.join(*path_parent), "w") as f:
        json.dump(metrics, f, cls = NumpyEncoder, indent=4)

103768993
1/1 [==============================] - 0s 47ms/step
Directory exists
Directory exists
Directory exists


1it [00:00,  7.74it/s]
1it [00:00,  9.84it/s]
1it [00:00,  9.08it/s]
1it [00:00,  9.46it/s]
1it [00:00,  9.49it/s]
1it [00:00,  9.60it/s]
1it [00:00,  9.66it/s]
1it [00:00,  9.41it/s]
1it [00:00,  9.69it/s]
1it [00:00,  9.32it/s]
1it [00:00,  9.58it/s]
1it [00:00,  9.42it/s]
1it [00:00,  8.65it/s]
1it [00:00,  8.93it/s]
1it [00:00,  8.52it/s]
1it [00:00,  9.12it/s]
1it [00:00,  9.61it/s]
1it [00:00,  9.34it/s]
1it [00:00,  9.67it/s]
1it [00:00,  9.61it/s]
1it [00:00,  9.45it/s]
1it [00:00,  9.60it/s]
1it [00:00,  9.77it/s]
1it [00:00,  9.57it/s]
1it [00:00,  9.56it/s]
1it [00:00,  9.30it/s]
1it [00:00,  9.77it/s]
1it [00:00,  9.46it/s]
1it [00:00,  9.90it/s]
1it [00:00,  9.57it/s]
1it [00:00,  9.76it/s]
1it [00:00,  9.65it/s]
1it [00:00,  9.55it/s]
1it [00:00,  9.29it/s]
1it [00:00,  9.75it/s]
1it [00:00,  7.90it/s]
1it [00:00,  9.54it/s]
1it [00:00,  9.44it/s]
1it [00:00,  9.53it/s]
1it [00:00,  9.63it/s]
1it [00:00,  9.54it/s]
1it [00:00,  9.37it/s]
1it [00:00,  9.39it/s]
1it [00:00,

95418342
1/1 [==============================] - 0s 44ms/step
Directory exists
Directory exists
Directory exists


1it [00:00,  7.20it/s]
1it [00:00,  9.51it/s]
1it [00:00,  9.26it/s]
1it [00:00,  9.27it/s]
1it [00:00,  9.03it/s]
1it [00:00,  9.73it/s]
1it [00:00,  9.76it/s]
1it [00:00,  9.52it/s]
1it [00:00,  9.58it/s]
1it [00:00,  9.52it/s]
1it [00:00,  9.02it/s]
1it [00:00,  9.54it/s]
1it [00:00,  9.66it/s]
1it [00:00,  9.26it/s]
1it [00:00,  9.04it/s]
1it [00:00,  9.13it/s]
1it [00:00,  9.23it/s]
1it [00:00,  9.55it/s]
1it [00:00,  9.78it/s]
1it [00:00,  9.34it/s]
1it [00:00,  9.62it/s]
1it [00:00,  9.65it/s]
1it [00:00,  9.59it/s]
1it [00:00,  9.08it/s]
1it [00:00,  9.29it/s]
1it [00:00,  9.25it/s]
1it [00:00,  9.40it/s]
1it [00:00,  9.79it/s]
1it [00:00,  9.60it/s]
1it [00:00,  9.29it/s]
1it [00:00,  9.62it/s]
1it [00:00,  9.87it/s]
1it [00:00,  9.52it/s]
1it [00:00,  9.61it/s]
1it [00:00, 10.11it/s]
1it [00:00, 10.01it/s]
1it [00:00,  9.70it/s]
1it [00:00,  9.53it/s]
1it [00:00,  9.22it/s]
1it [00:00,  9.73it/s]
1it [00:00,  9.91it/s]
1it [00:00,  9.64it/s]
1it [00:00,  9.52it/s]
1it [00:00,

105180865
1/1 [==============================] - 0s 50ms/step
Directory exists
Directory exists
Directory exists


...

1it [00:00,  7.03it/s]
1it [00:00,  9.57it/s]
1it [00:00,  8.87it/s]
1it [00:00,  7.65it/s]
1it [00:00,  7.92it/s]
1it [00:00,  8.20it/s]
1it [00:00,  8.25it/s]
1it [00:00,  8.76it/s]
1it [00:00,  9.30it/s]
1it [00:00,  9.31it/s]
1it [00:00,  9.25it/s]
1it [00:00,  9.62it/s]
1it [00:00,  9.58it/s]
1it [00:00,  9.48it/s]
1it [00:00,  9.14it/s]
1it [00:00,  9.41it/s]
1it [00:00,  9.65it/s]
1it [00:00,  9.60it/s]
1it [00:00,  9.09it/s]
1it [00:00,  9.65it/s]
1it [00:00,  9.36it/s]
1it [00:00,  9.33it/s]
1it [00:00,  9.42it/s]
1it [00:00,  9.25it/s]
1it [00:00,  9.50it/s]
1it [00:00,  8.88it/s]
1it [00:00,  9.58it/s]
1it [00:00,  9.27it/s]
1it [00:00,  9.68it/s]
1it [00:00,  9.71it/s]
1it [00:00,  9.69it/s]
1it [00:00,  9.75it/s]
1it [00:00,  9.34it/s]
1it [00:00,  9.66it/s]
1it [00:00,  9.83it/s]
1it [00:00,  9.58it/s]
1it [00:00,  9.50it/s]
1it [00:00,  9.06it/s]
1it [00:00,  9.37it/s]
1it [00:00,  9.42it/s]
1it [00:00,  9.72it/s]
1it [00:00,  9.47it/s]
1it [00:00,  9.29it/s]
1it [00:00,

27551681
1/1 [==============================] - 0s 60ms/step
Directory exists
Directory exists
Directory exists


1it [00:00,  7.66it/s]
1it [00:00,  9.81it/s]
1it [00:00,  9.77it/s]
1it [00:00,  9.59it/s]
1it [00:00,  9.58it/s]
1it [00:00,  9.77it/s]
1it [00:00, 10.06it/s]
1it [00:00,  9.60it/s]
1it [00:00,  9.31it/s]
1it [00:00,  9.25it/s]
1it [00:00,  9.50it/s]
1it [00:00,  9.49it/s]
1it [00:00,  9.06it/s]
1it [00:00,  9.70it/s]
1it [00:00,  9.04it/s]
1it [00:00,  9.32it/s]
1it [00:00,  9.65it/s]
1it [00:00,  9.23it/s]
1it [00:00,  9.40it/s]
1it [00:00,  9.47it/s]
1it [00:00,  9.45it/s]
1it [00:00,  9.46it/s]
1it [00:00,  9.55it/s]
1it [00:00,  9.63it/s]
1it [00:00,  9.43it/s]
1it [00:00,  9.13it/s]
1it [00:00,  9.24it/s]
1it [00:00,  8.91it/s]
1it [00:00,  9.12it/s]
1it [00:00,  9.24it/s]
1it [00:00,  9.23it/s]
1it [00:00,  9.38it/s]
1it [00:00,  9.57it/s]
1it [00:00,  9.55it/s]
1it [00:00,  9.54it/s]
1it [00:00,  9.54it/s]
1it [00:00,  9.42it/s]
1it [00:00,  9.23it/s]
1it [00:00,  9.07it/s]
1it [00:00,  9.43it/s]
1it [00:00,  9.19it/s]
1it [00:00,  8.88it/s]
1it [00:00,  9.22it/s]
1it [00:00,

344498881
1/1 [==============================] - 0s 67ms/step
Directory exists
Directory exists
Directory exists


1it [00:00,  7.73it/s]
1it [00:00,  9.75it/s]
1it [00:00,  9.19it/s]
1it [00:00,  9.36it/s]
1it [00:00,  9.60it/s]
1it [00:00,  9.74it/s]
1it [00:00,  9.83it/s]
1it [00:00,  9.62it/s]
1it [00:00,  9.44it/s]
1it [00:00,  9.28it/s]
1it [00:00,  8.61it/s]
1it [00:00,  9.42it/s]
1it [00:00,  8.74it/s]
1it [00:00, 10.09it/s]
1it [00:00,  8.52it/s]
1it [00:00,  9.71it/s]
1it [00:00, 10.09it/s]
1it [00:00,  9.42it/s]
1it [00:00,  9.86it/s]
1it [00:00, 10.01it/s]
1it [00:00,  9.27it/s]
1it [00:00,  9.23it/s]
1it [00:00,  9.58it/s]
1it [00:00,  9.47it/s]
1it [00:00,  9.47it/s]
1it [00:00,  9.26it/s]
1it [00:00,  9.46it/s]
1it [00:00,  9.33it/s]
1it [00:00,  9.60it/s]
1it [00:00,  9.46it/s]
1it [00:00,  9.39it/s]
1it [00:00,  9.69it/s]
1it [00:00,  9.83it/s]
1it [00:00,  9.83it/s]
1it [00:00,  9.90it/s]
1it [00:00,  7.78it/s]
1it [00:00,  9.72it/s]
1it [00:00,  9.47it/s]
1it [00:00,  9.80it/s]
1it [00:00,  9.81it/s]
1it [00:00,  9.40it/s]
1it [00:00,  8.92it/s]
1it [00:00,  9.65it/s]
1it [00:00,

# TF EVALUATION

In [5]:
loader = ProstateLesionDetectionUtils.DatasetsLoaders.Nifti_Loader.LoadSitkImages(ptdc = datasets["ProstateNet_Not_Integerized"], folder = prostatenet)
patients = loader.load_npobj()

In [10]:
def PrepareTraining(pat, seq_ls =None):
    dataset, labels = {}, {}
    for key in pat.keys():
        ds = []
        for item in seq_ls:
            ds.append(pat[key][item])
        dataset.update({key:np.transpose(np.stack(ds),(1,2,3,0))}) # tf.convert_to_tensor

    
        labels.update({key:np.expand_dims(pat[key]["Lesions"], axis = -1)}) # tf.convert_to_tensor
    return dataset, labels
dataset, labels = PrepareTraining(patients, seq_ls=["T2", "ADC", "DWI"])

x_train = {key:dataset[key] for key in dataset.keys() if key not in ValidationFolds["Fold 0"]}
y_train = {key:labels[key] for key in labels.keys() if key not in ValidationFolds["Fold 0"]}

x_test = {key:dataset[key] for key in dataset.keys() if key in ValidationFolds["Fold 0"]}
y_test = {key:labels[key] for key in labels.keys() if key in ValidationFolds["Fold 0"]}
Xtest = np.array(list(x_test.values()))
Ytest = np.array(list(y_test.values()))

In [11]:
import tensorflow_addons as tfa
model_names = ['Unet', 'Attention_Unet', 'Vnet', 'USE_net', 'Dense2_Unet', 'Trans_Unet']
base_path = "EXPERIMENTS_V0.2"
fold = "fold_0"
weights = "Final_1024.tf"
weights_check = "Checkpoint_1024.tf"


for md_nm in model_names:
    
    path_parent = [base_path, md_nm, fold]
    paths = [base_path, md_nm, fold,"Models", weights_check] # chains the paths
    model = tf.keras.models.load_model(os.path.join(*paths),compile = False)
    print(model.count_params())

    eval_params = {
        "MODEL": model,
        "EVALUATION_DATA": patients_test,# Needs to be a dictionary {PatName:{T2:arr, ADC:arr, DWI:arr, "Lesion":arr}}
        "SAVE_FOLDER":os.path.join(*path_parent)
    }
    STEPS_PER_EPOCH = len(Xtest) // 2
    INIT_LR = 0.0001
    MAX_LR = 0.05
    CLR = tfa.optimizers.CyclicalLearningRate(initial_learning_rate=INIT_LR,
    maximal_learning_rate=MAX_LR,
    scale_fn=lambda x: 1/(2.**(x-1)),
    step_size=2 * STEPS_PER_EPOCH)
    LOSS = ProstateLesionDetectionUtils.Evaluation.Loss_functions.combined_loss#tfa.losses.sigmoid_focal_crossentropy  #tfa.losses.sigmoid_focal_crossentropy
    OPTIMIZER = tf.keras.optimizers.legacy.Adam(CLR)
    METRICS = [ProstateLesionDetectionUtils.Evaluation.Loss_functions.dice_coefficient]
    eval_params["MODEL"].compile(optimizer=OPTIMIZER,
    loss=LOSS,
    metrics=METRICS)
    print(f"Model {md_nm}",eval_params["MODEL"].evaluate(Xtest, Ytest, batch_size=2))
 

103768993
35/35 [==============================] - 9s 114ms/step - loss: 0.5332 - dice_coefficient: 0.2754
Model Unet [0.5332456827163696, 0.2754172384738922]
95418342
35/35 [==============================] - 8s 135ms/step - loss: 0.6201 - dice_coefficient: 0.3009
Model Attention_Unet [0.6200505495071411, 0.30094581842422485]
105180865
35/35 [==============================] - 7s 137ms/step - loss: 0.5939 - dice_coefficient: 0.3209
Model Vnet [0.5938661694526672, 0.32087013125419617]
105620985
35/35 [==============================] - 6s 136ms/step - loss: 0.5793 - dice_coefficient: 0.3297
Model USE_net [0.5793249607086182, 0.32969093322753906]
27551681
35/35 [==============================] - 16s 204ms/step - loss: 0.5831 - dice_coefficient: 0.1990
Model Dense2_Unet [0.5830543637275696, 0.19895775616168976]
344498881
35/35 [==============================] - 7s 145ms/step - loss: 0.4837 - dice_coefficient: 0.3515
Model Trans_Unet [0.48366978764533997, 0.35150447487831116]
